In [2]:
import pandas as pd

from evidently.v2.metrics import Metric
from evidently.v2.datasets import DataDefinition
from evidently.v2.datasets import Dataset
from evidently.v2.metrics import SingleValue
from evidently.v2.metrics import SingleValueCheck

In [5]:
from evidently.v2.checks.numerical_checks import le, ge
from typing import Optional
from typing import List
from evidently.renderers.html_widgets import plotly_figure
from plotly.express import line

# user-friendly interface
def max_metric(column_name: str, checks: Optional[List[SingleValueCheck]] = None, group_by: Optional[str] = None) -> Metric:
    return MaxMetric(column_name, checks)


# implementation
class MaxMetric(Metric[SingleValue]):
    _column_name: str
    
    def __init__(self, column_name: str, checks: Optional[List[SingleValueCheck]] = None):
        self._column_name = column_name
        self._checks = checks if checks is not None else [le(10), ge(6)]

    def calculate(self, current_data: Dataset, reference_data: Optional[Dataset]) -> SingleValue:
        x = current_data.column(self._column_name)
        value = x.max()
        result = SingleValue(value=value)
        figure = line(x)
        figure.add_hrect(6, 10)
        #result.set_widget([plotly_figure(title=self.display_name(), figure=figure)])
        return result

    def display_name(self) -> str:
        return f"Max value for {self._column_name}"

# usage example
data = pd.DataFrame(data={"column_1": [1, 2, 3, 4, -1, 5]})

result = max_metric("column_1", checks=None).call(Dataset.from_pandas(data, data_definition=DataDefinition()), None)
result.show()